<a href="https://colab.research.google.com/github/H4rfu1/ML-project-TokoLitik-tim-Undagi/blob/main/Response/MainProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Required Library

In [ ]:
import pandas as pd
import tensorflow as tf
import zipfile

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load Dataset

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hgultekin/bbcnewsarchive

bbcnewsarchive.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

 bbcnewsarchive.zip   gdrive		 kaggle.json
 files		     'kaggle (1).json'	 sample_data


In [ ]:
zip_ref = zipfile.ZipFile('bbcnewsarchive.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [ ]:
!ls

 bbcnewsarchive.zip   gdrive		 kaggle.json
 files		     'kaggle (1).json'	 sample_data


In [ ]:
#sumber dataset https://www.kaggle.com/hgultekin/bbcnewsarchive?select=bbc-news-data.csv

df = pd.read_csv('files/bbc-news-data.csv',sep='\t')

# Check Dataset

In [ ]:
#to check shape of the dataset
print(df.shape) 

#to look the header of dataset
df.head()


(2225, 4)


,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [ ]:
#drop unecesary column
df = df.drop(columns=['filename'])
df = df.drop(columns=['title'])

#separate column category and add value to it
category = pd.get_dummies(df.category)
df_new = pd.concat([df, category], axis=1)
df_new = df_new.drop(columns='category')
df_new

,content,business,entertainment,politics,sport,tech
0,Quarterly profits at US media giant TimeWarne...,1,0,0,0,0
1,The dollar has hit its highest level against ...,1,0,0,0,0
2,The owners of embattled Russian oil giant Yuk...,1,0,0,0,0
3,British Airways has blamed high fuel prices f...,1,0,0,0,0
4,Shares in UK drinks and food firm Allied Dome...,1,0,0,0,0
...,...,...,...,...,...,...
2220,BT is introducing two initiatives to help bea...,0,0,0,0,1
2221,Computer users across the world continue to i...,0,0,0,0,1
2222,A new European directive could put software w...,0,0,0,0,1
2223,The man making sure US computer networks are ...,0,0,0,0,1


# Peprocess Dataset

In [ ]:
#remove punctuations
import string
def punctuation(sentence):
    review = [letters.lower() for letters in sentence if letters not in string.punctuation ]
    review = ''.join(review)
    return review

df['review'] = df['review'].apply(punctuation)

,content,business,entertainment,politics,sport,tech
0,Quarterly profits at US media giant TimeWarne...,1,0,0,0,0
1,The dollar has hit its highest level against ...,1,0,0,0,0
2,The owners of embattled Russian oil giant Yuk...,1,0,0,0,0
3,British Airways has blamed high fuel prices f...,1,0,0,0,0
4,Shares in UK drinks and food firm Allied Dome...,1,0,0,0,0
...,...,...,...,...,...,...
2220,BT is introducing two initiatives to help bea...,0,0,0,0,1
2221,Computer users across the world continue to i...,0,0,0,0,1
2222,A new European directive could put software w...,0,0,0,0,1
2223,The man making sure US computer networks are ...,0,0,0,0,1


# change data type

In [ ]:
#get label of dataset
content = df_new['content'].values
label = df_new[['business', 'entertainment', 'politics', 'sport', 'tech']].values

# Build Train and test sets

In [ ]:
content_train, content_test, label_train, label_test = train_test_split(content, label, test_size=0.2)

# Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token='OOV')
tokenizer.fit_on_texts(content_train) 
 
sekuens_train = tokenizer.texts_to_sequences(content_train)
sekuens_test = tokenizer.texts_to_sequences(content_test)
 
padded_train = pad_sequences(sekuens_train, padding='post', maxlen=200) 
padded_test = pad_sequences(sekuens_test, padding='post', maxlen=200)

# Build Model

In [ ]:
#build model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Callback

In [ ]:
#to stop train while acuracy greather than 90%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.9 and logs.get('val_accuracy') > 0.9):
      print("\nAccuracy and val accuracy has reach >90%!")
      self.model.stop_training = True
callbacks = myCallback()

# Train

In [ ]:
num_epochs = 150
history = model.fit(padded_train, label_train, epochs=num_epochs, callbacks=[callbacks],
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/150
56/56 - 6s - loss: 1.5828 - accuracy: 0.2528 - val_loss: 1.5671 - val_accuracy: 0.2157
Epoch 2/150
56/56 - 6s - loss: 1.4111 - accuracy: 0.3472 - val_loss: 1.1838 - val_accuracy: 0.4921
Epoch 3/150
56/56 - 6s - loss: 1.2176 - accuracy: 0.5101 - val_loss: 1.1672 - val_accuracy: 0.4854
Epoch 4/150
56/56 - 6s - loss: 0.9357 - accuracy: 0.5978 - val_loss: 1.0607 - val_accuracy: 0.5416
Epoch 5/150
56/56 - 6s - loss: 0.7120 - accuracy: 0.7051 - val_loss: 0.8560 - val_accuracy: 0.6787
Epoch 6/150
56/56 - 6s - loss: 0.6199 - accuracy: 0.7528 - val_loss: 0.8041 - val_accuracy: 0.6629
Epoch 7/150
56/56 - 6s - loss: 0.3962 - accuracy: 0.8556 - val_loss: 0.8213 - val_accuracy: 0.6921
Epoch 8/150
56/56 - 6s - loss: 0.3075 - accuracy: 0.9011 - val_loss: 1.0578 - val_accuracy: 0.6944
Epoch 9/150
56/56 - 6s - loss: 0.2339 - accuracy: 0.9174 - val_loss: 0.8755 - val_accuracy: 0.7326
Epoch 10/150
56/56 - 6s - loss: 0.1175 - accuracy: 0.9674 - val_loss: 0.8166 - val_accuracy: 0.7865
Epoch 11/